# Day 18
## Part 1

Doing a complete breadth first search looks like it might be intractable, so first create a graph of the shortest distance from each point of interest to each other point that's accessible without passing through another door or key.  

In [1]:
from collections import defaultdict, deque

def draw_graph(vault):
    def bfs(row, col):
        seen = {(row, col)}
        search = deque([(row, col, 0)])
        accessible_points = {}
        
        while search:
            r, c, steps = search.popleft()
            
            for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                new_r = r + dr
                new_c = c + dc
                pos = vault[new_r][new_c]
                if (new_r, new_c) not in seen:
                    if pos in '.@':
                        search.append((new_r, new_c, steps + 1))
                        seen.add((new_r, new_c))
                    elif pos != '#':
                        accessible_points[pos] = steps + 1
                        
        return accessible_points
    
    return {
        vault[r][c]: bfs(r, c)
        for r, row in enumerate(vault)
        for c, pos in enumerate(row)
        if pos not in '#.'
    }

In [2]:
test_vault_1 = '''#########
#b.A.@.a#
#########'''.splitlines()

draw_graph(test_vault_1)

{'b': {'A': 2}, 'A': {'b': 2, 'a': 4}, '@': {'a': 2, 'A': 2}, 'a': {'A': 4}}

In [3]:
test_vault_2 = '''########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################
'''.splitlines()

draw_graph(test_vault_2)

{'f': {'D': 2},
 'D': {'E': 2, 'f': 2},
 'E': {'e': 2, 'D': 2},
 'e': {'C': 2, 'E': 2},
 'C': {'b': 2, 'e': 2},
 'b': {'A': 2, 'C': 2},
 'A': {'b': 2, 'a': 4},
 '@': {'a': 2, 'A': 2},
 'a': {'B': 2, 'A': 4},
 'B': {'c': 2, 'a': 2},
 'c': {'B': 2, 'd': 24},
 'd': {'c': 24}}

Now do an exhaustive search on these graphs to find the shortest legitimate path, where the doors are unlocked. Select the shortest path so far at each point, and prune any branches longer than the best found so far. 

In [4]:
import heapq
import math
from collections import namedtuple
from pyrsistent import pset

def collect_keys(graph):
    all_keys = {k for k in graph if k.islower()}
    shortest_path = math.inf
    seen = {}
    
    # Each search state is a tuple of the number of
    # steps taken, the node we're at, and the keys
    # collected
    search = [(0, '@', pset())]
    
    while search:
        steps, node, keys = heapq.heappop(search)
        
        for next_node in graph[node]:
            # Update the length of the path taken
            new_steps = steps + graph[node][next_node]
            
            if new_steps < shortest_path:
                if next_node.islower():
                    new_keys = keys.add(next_node)
                    # Have we found all the keys?
                    if len(new_keys) == len(all_keys):
                        shortest_path = new_steps
                    else:
                        # Check there isn't a shorter way to get here
                        # with these keys
                        if seen.get((next_node, new_keys), math.inf) > new_steps:
                            seen[(next_node, new_keys)] = new_steps
                            heapq.heappush(search, (new_steps, next_node, new_keys))
                elif next_node.lower() in keys:
                    if seen.get((next_node, keys), math.inf) > new_steps:
                        seen[(next_node, keys)] = new_steps
                        heapq.heappush(search, (new_steps, next_node, keys))
                    
    return shortest_path
    

def part_1(vault):
    return collect_keys(draw_graph(vault))


assert part_1(test_vault_1) == 8

In [5]:
assert part_1(test_vault_2) == 86

In [6]:
test_vault_3 = '''########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################'''.splitlines()

assert part_1(test_vault_3) == 132

In [7]:
test_vault_4 = '''#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################'''.splitlines()

assert part_1(test_vault_4) == 136

That had a good think about things.

In [8]:
test_vault_5 = '''########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################
'''.splitlines()

assert part_1(test_vault_5) == 81

In [9]:
vault = open('input').read().splitlines()

In [10]:
%%timeit

part_1(vault)

591 ms ± 7.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Part 2

In [11]:
import itertools

def draw_graph_2(vault):
    def bfs(vlt, row, col):
        seen = {(row, col)}
        search = deque([(row, col, 0)])
        accessible_points = {}
        
        while search:
            r, c, steps = search.popleft()
            
            for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                new_r = r + dr
                new_c = c + dc
                pos = vlt[new_r][new_c]
                if (new_r, new_c) not in seen:
                    if pos in '.1234':
                        search.append((new_r, new_c, steps + 1))
                        seen.add((new_r, new_c))
                    elif pos != '#':
                        accessible_points[pos] = steps + 1
                        
        return accessible_points
    
    c = itertools.count(1)
    v = [[str(next(c)) if x == '@' else x for x in row]
         for row in vault]
    
    return {
        v[r][c]: bfs(v, r, c)
        for r, row in enumerate(v)
        for c, pos in enumerate(row)
        if pos not in '#.'
    }

In [12]:
test_vault_6 = '''#######
#a.#Cd#
##@#@##
#######
##@#@##
#cB#.b#
#######'''.splitlines()

draw_graph_2(test_vault_6)

{'a': {},
 'C': {'d': 1},
 'd': {'C': 1},
 '1': {'a': 2},
 '2': {'C': 1},
 '3': {'B': 1},
 '4': {'b': 2},
 'c': {'B': 1},
 'B': {'c': 1},
 'b': {}}

In [51]:
from collections import deque


def collect_keys_2(graph):
    all_keys = {k for k in graph if k.islower()}
    shortest_path = math.inf
    seen = {}
    
    # Each search state is a tuple of the number of
    # steps taken, the node we're at, and the keys
    # collected
    search = deque([(0, pset({'1', '2', '3', '4'}), pset())])
    
    while search:
        #print(search)
        steps, nodes, keys = search.popleft()
        
        for node in nodes:
            for next_node in graph[node]:
                # Update the length of the path taken
                new_steps = steps + graph[node][next_node]
                new_nodes = nodes.discard(node).add(next_node)

                if new_steps < shortest_path:
                    if next_node.islower():
                        new_keys = keys.add(next_node)
                        # Have we found all the keys?
                        if len(new_keys) == len(all_keys):
                            shortest_path = new_steps
                        else:
                            # Check there isn't a shorter way to get here
                            # with these keys
                            if seen.get((new_nodes, new_keys), math.inf) > new_steps:
                                seen[(new_nodes, new_keys)] = new_steps
                                search.append((new_steps, new_nodes, new_keys))
                    elif next_node.lower() in keys:
                        if seen.get((new_nodes, keys), math.inf) > new_steps:
                            seen[(new_nodes, keys)] = new_steps
                            search.append((new_steps, new_nodes, keys))
                    
    return shortest_path


def part_2(vault):
    return(collect_keys_2(draw_graph_2(vault)))


assert part_2(test_vault_6) == 8

In [52]:
test_vault_7 = '''###############
#d.ABC.#.....a#
######@#@######
###############
######@#@######
#b.....#.....c#
###############'''.splitlines()

assert part_2(test_vault_7) == 24

In [53]:
test_vault_8 = '''#############
#DcBa.#.GhKl#
#.###@#@#I###
#e#d#####j#k#
###C#@#@###J#
#fEbA.#.FgHi#
#############'''.splitlines()

assert part_2(test_vault_8) == 32

In [54]:
%%timeit

test_vault_9 = '''#############
#g#f.D#..h#l#
#F###e#E###.#
#dCba@#@BcIJ#
#############
#nK.L@#@G...#
#M###N#H###.#
#o#m..#i#jk.#
#############'''.splitlines()

assert part_2(test_vault_9) == 72

810 ms ± 5.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
vault_2 = open('input_2').read().splitlines()

In [68]:
%%timeit

part_2(vault_2)

6min 3s ± 899 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Print the answer you idiot.

In [69]:
part_2(vault_2)

2086

That takes too long. I've thought of a potentially quicker way of doing it (a key to key graph that tracks the doors inbetween) but I'm not sure how robust it will be.